In [8]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from ipywidgets import interactive
from IPython.display import Audio, display
import os
from ipynb.fs.full.dateSelector import year, month, day, distance, site, getDateTime
from ipynb.fs.full.gaussianModel import gaussian, gaussiany, getEventFrame
from ipynb.fs.full.siteData import sites, getSiteData
from random import randrange
from __future__ import print_function
from ipyleaflet import (Map, Marker, MarkerCluster, TileLayer, ImageOverlay,
                        Polyline, Polygon, Rectangle, Circle, CircleMarker,
                        GeoJSON, DrawControl, basemaps)

matplotlib.rcParams['figure.figsize'] = [20.0, 12.0]

def normalise(df):
    return (df - df.mean()) / (df.max() - df.min())

def toDateTime(timeString):
    return datetime.strptime(timeString, '%d/%m/%Y %H:%M:%S').timestamp()

df = pd.read_csv('data/events.csv')
df['datetime'] = df['DateTime(IST)'].apply(lambda x: toDateTime(x))
df1 = df[['datetime', 'DateTime(IST)', 'Latitude', 'Longitude', 'Magnitude']]

In [9]:
import geopy.distance

coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print(geopy.distance.vincenty(coords_1, coords_2).km)


279.35290160386563


In [10]:
from ipynb.fs.full.dateSelector import year, month, day, getDateTime

matplotlib.rcParams['figure.figsize'] = [20.0, 12.0]
class event:
    def __init__(self):
        center = [19, 72]
        zoom = 3
        self.m = Map(center=center, zoom=zoom)
        display(self.m)
        
        self.circles = [Circle(location=center, radius=1000)]
        for circle in self.circles:
            self.m.add_layer(circle)

    def update(self, year, month, day, distance, site):
        start, stop = getDateTime(year, month, day, distance, site)
        for i in sites:
            if i['site'] == site:
                siteLat = i['lat']
                siteLng = i['lng']
                siteSrNo = i['srNo']
                break
        
        #self.m.remove_layer(self.marker_cluster)
        for circle in self.circles:
            self.m.remove_layer(circle)
        startYear = datetime(year=start.year, month=1, day=1)
        stopYear = startYear + timedelta(days = 365)
        numEventsThisYear = len(df1.loc[(df1['datetime'] >= startYear.timestamp())
                                        & (df1['datetime'] <= stopYear.timestamp())])
        print('events in year {0} : {1}'.format(start.year, numEventsThisYear))
        startMonth = datetime(year=start.year, month=start.month, day=1)
        stopMonth = startMonth + timedelta(days = 30)
        numEventsThisMonth = len(df1.loc[(df1['datetime'] >= startMonth.timestamp())
                                        & (df1['datetime'] <= stopMonth.timestamp())])
        print('events in month {0} : {1}'.format(start.month, numEventsThisMonth))
        df2 = df1.loc[(df1['datetime'] >= start.timestamp()) &
                      (df1['datetime'] <= stop.timestamp())]
        locations = [[df2.iloc[row]['Latitude'], df2.iloc[row]['Longitude']]
                     for row in range(len(df2))]
        magnitudes = [df2.iloc[row]['Magnitude'] for row in range(len(df2))]
        
        print('events in duration: {0}'.format(len(df2)))
        self.circles = [Circle(location=location, radius=int(magnitude*10000)) 
                        for location, magnitude  in zip(locations, magnitudes)]
        for circle in self.circles:
            self.m.add_layer(circle)
        dfList = []
        for index, row in df2.iterrows():
            eventDistance = geopy.distance.vincenty((row['Latitude'], row['Longitude']),
                                                    (siteLat, siteLng)).km
            if eventDistance > distance[0] and eventDistance < distance[1]:
                label = '{0}:{1}'.format(randrange(255), eventDistance)
                dfList.append(getEventFrame(row['datetime'], row['Magnitude'], label = label))
        if len(dfList):
            df2 = pd.concat(dfList) 
            dfSite = pd.concat([getSiteData(start.timestamp(),
                                            stop.timestamp(), siteList=[siteSrNo]), df2])
            df3 = dfSite.sort_values('datetime', ascending=True)
            df4 = df3.interpolate()
            
            df4.plot(x = 'datetime') 
        
        
    
evnt = event()

w = interactive(evnt.update, year=year, month=month, day=day, distance=distance, site=site)
display(w)

Map(basemap={'max_zoom': 19, 'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'attribution': 'Map …

interactive(children=(Dropdown(description='Year:', index=2, options=(2016, 2017, 2018, 2019, 2020, 2014, 2015…